In [1]:
import requests
import os 
import functools
import json
from dotenv import load_dotenv, find_dotenv
from decouple import config
from requests.auth import HTTPBasicAuth
from jira import  JIRA
import pandas as pd 
load_dotenv(find_dotenv())

username = os.environ.get('JIRA_USER_LOCAL')
pwd = os.environ.get('JIRA_TOKEN_LOCAL')
jira_url = os.environ.get('BASE_URL')

print(f'Jira_Url: {jira_url}')
jira = JIRA(basic_auth=(username, pwd), options={'server':jira_url, 'verify':False}, validate=False)

Jira_Url: https://alightdevelopmentandit.atlassian.net/


d:\Django_projects\api_env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alightdevelopmentandit.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
# headers = {"Accept": "application/json",
#             "Content-Type": "application/json",
#             "X-Atlassian-Token": "nocheck"}
# auth = HTTPBasicAuth(username, pwd)
# request_url = jira_url + 'rest/api/2/issue/ADAMIGR-67'
# print(f'request_url: {request_url}')
# # sys.exit(0)
# response = requests.get(request_url, headers=headers, auth=auth, verify=False)
# if response.status_code != 200:
#     print(f'Status code: {response.status_code}')
# else:
#     print('Got data valid response')
#     json_data = response.json()
# fields_dict = json_data['fields']

In [3]:
# functions


def rgetattr(obj, attr, *args):
    def __getattr(obj, attr):
        return getattr(obj, attr, *args)
    return functools.reduce(__getattr, [obj] + attr.split('.'))

def get_field_data(field, name):
    return rgetattr(field, name, 'Not Populated')

In [4]:
jql = 'project = ADAMIGR AND issuetype = Epic AND status = Open ORDER BY createdDate'
proj_issues = []

print('Extracting epics...')
for issue in jira.search_issues(jql, maxResults=50):
    # issue_list = issue.fields()
    # print(*issue_list, sep="\n")


    issue_func_dict = {
        'key': issue.key,
        'issue_id': issue.id,
        'project': issue.fields.project.name,
        'status': issue.fields.status.name,
        'issue_type': issue.fields.issuetype.name,
        'create_date': issue.fields.created,
        'update_date': issue.fields.updated,
        'summary': issue.fields.summary
    }
    # print('{}: {}'.format(issue.key, issue.fields.summary))
    issue_list = {}
    for key in issue_func_dict:
        f = issue_func_dict.get(key)
        res = f
        issue_list[key] = str(res)

        # print(f'result: {res}')
    proj_issues.append(issue_list)

print('All Migration Ticket Extraction complete.')


Extracting epics...


d:\Django_projects\api_env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alightdevelopmentandit.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Django_projects\api_env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alightdevelopmentandit.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


All Migration Ticket Extraction complete.


In [5]:
df_proj = pd.DataFrame.from_dict(proj_issues)
df_proj = df_proj[df_proj['issue_type'] == 'Epic']
df_proj["create_date"] = pd.to_datetime(
    df_proj["create_date"], format="%Y-%m-%d")
df_proj["update_date"] = pd.to_datetime(
    df_proj["update_date"], format="%Y-%m-%d")

df_proj

,key,issue_id,project,status,issue_type,create_date,update_date,summary
0,ADAMIGR-1363,133877,ADA Migration Requests,Open,Epic,2022-03-16 12:00:51.692000-05:00,2022-03-16 12:01:09.867000-05:00,ACE Release 2.8.1
1,ADAMIGR-1362,133873,ADA Migration Requests,Open,Epic,2022-03-16 11:59:32.351000-05:00,2022-03-16 11:59:45.418000-05:00,ACE Release 2.7.2
2,ADAMIGR-1361,133869,ADA Migration Requests,Open,Epic,2022-03-16 11:59:03.246000-05:00,2022-03-16 11:59:21.509000-05:00,ACE Release 2.7.1
3,ADAMIGR-1360,133867,ADA Migration Requests,Open,Epic,2022-03-16 11:58:29.901000-05:00,2022-03-16 11:58:49.542000-05:00,ACE Release 2.6.2
4,ADAMIGR-1359,133866,ADA Migration Requests,Open,Epic,2022-03-16 11:58:01.900000-05:00,2022-03-16 11:58:20.959000-05:00,ACE Release 2.6.1
5,ADAMIGR-1358,133864,ADA Migration Requests,Open,Epic,2022-03-16 11:57:31.462000-05:00,2022-03-16 11:57:48.666000-05:00,ACE Release 2.5.3
6,ADAMIGR-1357,133847,ADA Migration Requests,Open,Epic,2022-03-16 11:56:59.909000-05:00,2022-03-16 11:57:19.069000-05:00,ACE Release 2.5.2
7,ADAMIGR-1356,133843,ADA Migration Requests,Open,Epic,2022-03-16 11:56:14.397000-05:00,2022-03-16 11:56:44.914000-05:00,ACE Release 2.5.1
8,ADAMIGR-34,104030,ADA Migration Requests,Open,Epic,2022-02-11 07:20:21.529000-06:00,2022-03-23 02:25:27.717000-05:00,ACE Release 2.4.2
9,ADAMIGR-33,104029,ADA Migration Requests,Open,Epic,2022-02-11 07:19:17.600000-06:00,2022-03-23 03:29:03.378000-05:00,ACE Release 2.4.1


In [6]:
# def remove_empties_from_dict(a_dict):
#     new_dict = {}
#     for k, v in a_dict.items():
#         if isinstance(v, dict):
#             v = remove_empties_from_dict(v)
#         if v is not None:
#             new_dict[k] = v
#     return new_dict or None

# not_null_dict = remove_empties_from_dict(fields_dict)
# # print(not_null_dict['parent'])
# df_parent = pd.DataFrame(not_null_dict['parent'])
# df_parent

In [7]:
# extracting issues from tickets



epic_link_list = df_proj['key'].tolist()
print(epic_link_list)

epic_link_issues = []
validation_issues = []
print('Extracting issues...')
for epic_link in epic_link_list:
    jql = '"Epic Link"=' + epic_link
    for issue in jira.search_issues(jql, maxResults=50):
        # issue_list = issue.fields()        
        # print(*issue_list, sep="\n")
        issue_func_dict = {
            'key': get_field_data(issue, 'key'),
            'issue_id': get_field_data(issue, 'id'),
            'issue_parent_ticket': get_field_data(issue, 'fields.parent.key'),
            'issue_parent_ticket_summary': get_field_data(issue, 'fields.parent.fields.summary'),
            'issue_rest_url': get_field_data(issue, 'self'), #issue.self,
            'assignee': get_field_data(issue, 'fields.assignee.displayName'), #issue.fields.assignee.displayName,
            'reporter': get_field_data(issue, 'fields.reporter.displayName'), #issue.fields.reporter.displayName,
            'status': get_field_data(issue, 'fields.status.name'), #issue.fields.status.name,
            'issue_type': get_field_data(issue, 'fields.issuetype.name'), #issue.fields.issuetype.name,
            'resolution_description': get_field_data(issue, 'fields.status.description'), #issue.fields.status.description,
            'create_date': get_field_data(issue, 'fields.created'), #issue.fields.created,
            'update_date': get_field_data(issue, 'fields.updated'), #issue.fields.updated,
            'summary': get_field_data(issue, 'fields.summary'), #issue.fields.summary,
            'description': get_field_data(issue, 'fields.description'), #issue.fields.description
            'service_name': get_field_data(issue, 'fields.customfield_10320'), 
            'mig_req_url': get_field_data(issue, 'fields.customfield_10321'), 
            'dev_approved_by': get_field_data(issue, 'fields.customfield_10251'), 
            'qa_test_approved_by': get_field_data(issue, 'fields.customfield_10252'), 
            'int_test_approved_by': get_field_data(issue, 'fields.customfield_10327'), 
            'dev_approved': get_field_data(issue, 'fields.customfield_10234'), 
            'new_docker_secrets': get_field_data(issue, 'fields.customfield_10325.value'),
            'qa_test_script_approved': get_field_data(issue, 'fields.customfield_10237'), 
            'ace_migration_status': get_field_data(issue, 'fields.customfield_10238.value'),
            'migration_approved_by': get_field_data(issue, 'fields.customfield_10252.displayName.value'),  

            
        }
        # print('{}: {}'.format(issue.key, issue.fields.summary))
        issue_list = {}
        for key in issue_func_dict:
            f = issue_func_dict.get(key)
            res = f
            issue_list[key] = str(res)

            # print(f'result: {res}')
        epic_link_issues.append(issue_list)
      


print('Epic Link Issue Extraction complete.')


['ADAMIGR-1363', 'ADAMIGR-1362', 'ADAMIGR-1361', 'ADAMIGR-1360', 'ADAMIGR-1359', 'ADAMIGR-1358', 'ADAMIGR-1357', 'ADAMIGR-1356', 'ADAMIGR-34', 'ADAMIGR-33', 'ADAMIGR-9', 'ADAMIGR-8', 'ADAMIGR-6', 'ADAMIGR-2', 'ADAMIGR-1']
Extracting issues...


d:\Django_projects\api_env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alightdevelopmentandit.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Django_projects\api_env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alightdevelopmentandit.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Django_projects\api_env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alightdevelopmentandit.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warn

Epic Link Issue Extraction complete.


In [8]:

df = pd.DataFrame.from_dict(epic_link_issues)
df.tail(20)

,key,issue_id,issue_parent_ticket,issue_parent_ticket_summary,issue_rest_url,assignee,reporter,status,issue_type,resolution_description,...,service_name,mig_req_url,dev_approved_by,qa_test_approved_by,int_test_approved_by,dev_approved,new_docker_secrets,qa_test_script_approved,ace_migration_status,migration_approved_by
39,ADAMIGR-1232,131445,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Bhavik Shah,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-foundationwidgets-service,https://alightdevelopmentandit.atlassian.net/b...,Bipin Kalola,Dishant Soni,Dishant Soni,"[<JIRA CustomFieldOption: value='Yes', id='178...",No,"[<JIRA CustomFieldOption: value='Yes', id='178...",Ready for QA migration,Not Populated
40,ADAMIGR-1223,131436,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Bhavik Shah,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-wellnesswidgets-service,https://alightdevelopmentandit.atlassian.net/b...,Bhavik Shah,Amit Kumar Saini,Amit Kumar Saini,"[<JIRA CustomFieldOption: value='Yes', id='178...",No,"[<JIRA CustomFieldOption: value='Yes', id='178...",Ready for QA migration,Not Populated
41,ADAMIGR-1215,131423,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Vanita Miglani,In Progress,Migration Request,This issue is being actively worked on at the ...,...,authentication-service,https://alightdevelopmentandit.atlassian.net/b...,Vijayakumar G,Jesna Sajeev,Jesna Sajeev,"[<JIRA CustomFieldOption: value='Yes', id='178...",Yes,"[<JIRA CustomFieldOption: value='Yes', id='178...",Ready for QA migration,Not Populated
42,ADAMIGR-1204,131326,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Soham Mehta,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-financialeducationrevampwidgets-servic...,https://alightdevelopmentandit.atlassian.net/b...,Soham Mehta,urvish.pandya.2@alight.com,Preethi Narayan,"[<JIRA CustomFieldOption: value='Yes', id='178...",Not Populated,"[<JIRA CustomFieldOption: value='Yes', id='178...",Ready for QA migration,Not Populated
43,ADAMIGR-1198,131268,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Shubhra Tiwari,In Progress,Migration Request,This issue is being actively worked on at the ...,...,edv-base-ng13-app,https://alightdevelopmentandit.atlassian.net/b...,None,None,Shruti Nigam,None,Not Populated,None,Not Populated,Not Populated
44,ADAMIGR-1191,131155,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Amritpal Singh,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-iva-service,None,None,None,Vedant Sinha,None,Yes,None,Not Populated,Not Populated
45,ADAMIGR-1168,130644,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Chirag Shah,Chirag Shah,Closed,Migration Request,"The issue is considered finished, the resoluti...",...,channel-dcmanageinvestmentswidgets-service,https://alightdevelopmentandit.atlassian.net/b...,Het Shah,Anjali Gupta,Anjali Gupta,"[<JIRA CustomFieldOption: value='Yes', id='178...",No,"[<JIRA CustomFieldOption: value='Yes', id='178...",Not Populated,Not Populated
46,ADAMIGR-1167,130643,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Scott Holze,sivakami,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-definedbenefitretirement-service,https://alightdevelopmentandit.atlassian.net/b...,Matt Caudill,Monique Harvey,sivakami,"[<JIRA CustomFieldOption: value='Yes', id='178...",Not Populated,"[<JIRA CustomFieldOption: value='Yes', id='178...",Not Populated,Not Populated
47,ADAMIGR-1158,130431,ADAMIGR-33,ACE Release 2.4.1,https://alightdevel

In [9]:

# df = df[df['issue_type'] == 'Migration Request']
# df['create_date'] = pd.to_datetime(df['create_date'], utc=True)
# df["update_date"] = pd.to_datetime(df["update_date"], utc=True)

# df

In [10]:
def get_type(input_val):
    if input_val:
        if 'None' in input_val:
            return 'None'
        if 'Yes' in input_val:
            return 'Yes'
        if 'No' in input_val:
            return 'No'

df['dev_approved'] = df['dev_approved'].apply(get_type)
df['qa_test_script_approved'] = df['qa_test_script_approved'].apply(get_type)
df.tail(20)

,key,issue_id,issue_parent_ticket,issue_parent_ticket_summary,issue_rest_url,assignee,reporter,status,issue_type,resolution_description,...,service_name,mig_req_url,dev_approved_by,qa_test_approved_by,int_test_approved_by,dev_approved,new_docker_secrets,qa_test_script_approved,ace_migration_status,migration_approved_by
39,ADAMIGR-1232,131445,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Bhavik Shah,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-foundationwidgets-service,https://alightdevelopmentandit.atlassian.net/b...,Bipin Kalola,Dishant Soni,Dishant Soni,Yes,No,Yes,Ready for QA migration,Not Populated
40,ADAMIGR-1223,131436,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Bhavik Shah,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-wellnesswidgets-service,https://alightdevelopmentandit.atlassian.net/b...,Bhavik Shah,Amit Kumar Saini,Amit Kumar Saini,Yes,No,Yes,Ready for QA migration,Not Populated
41,ADAMIGR-1215,131423,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Vanita Miglani,In Progress,Migration Request,This issue is being actively worked on at the ...,...,authentication-service,https://alightdevelopmentandit.atlassian.net/b...,Vijayakumar G,Jesna Sajeev,Jesna Sajeev,Yes,Yes,Yes,Ready for QA migration,Not Populated
42,ADAMIGR-1204,131326,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Soham Mehta,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-financialeducationrevampwidgets-servic...,https://alightdevelopmentandit.atlassian.net/b...,Soham Mehta,urvish.pandya.2@alight.com,Preethi Narayan,Yes,Not Populated,Yes,Ready for QA migration,Not Populated
43,ADAMIGR-1198,131268,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Shubhra Tiwari,In Progress,Migration Request,This issue is being actively worked on at the ...,...,edv-base-ng13-app,https://alightdevelopmentandit.atlassian.net/b...,None,None,Shruti Nigam,None,Not Populated,None,Not Populated,Not Populated
44,ADAMIGR-1191,131155,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Amritpal Singh,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-iva-service,None,None,None,Vedant Sinha,None,Yes,None,Not Populated,Not Populated
45,ADAMIGR-1168,130644,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Chirag Shah,Chirag Shah,Closed,Migration Request,"The issue is considered finished, the resoluti...",...,channel-dcmanageinvestmentswidgets-service,https://alightdevelopmentandit.atlassian.net/b...,Het Shah,Anjali Gupta,Anjali Gupta,Yes,No,Yes,Not Populated,Not Populated
46,ADAMIGR-1167,130643,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Scott Holze,sivakami,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-definedbenefitretirement-service,https://alightdevelopmentandit.atlassian.net/b...,Matt Caudill,Monique Harvey,sivakami,Yes,Not Populated,Yes,Not Populated,Not Populated
47,ADAMIGR-1158,130431,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Soham Mehta,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-dcpaymenthistorywidgets-service (https...,https://alightdevelopmentandit.atlassian.net/b...,Soham Mehta,urvish.pandya.2@alight.com,Akash Gupta,Yes,Not Populated,Yes,Ready for QA migration,Not Populated
48,ADAMIGR-1155,130421,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Garima Sharma,Gunjan Goswami,Open,Migration Request,The issue is open and ready for the assignee t...,...,None,https:

In [11]:
# # For making a json file using python dictionary 
bus_rule_dict = {

    "qa": ['int_test_approved_by','dev_approved_by', 'dev_approved'],

    "qc": ['int_test_approved_by','dev_approved_by', 'dev_approved', 'qa_test_script_approved', 'qa_test_approved_by', 'migration_approved_by'],

    "ste":['int_test_approved_by','dev_approved_by','dev_approved','qa_test_script_approved','migration_approved_by','qa_test_approved_by'],

    "prod":['int_test_approved_by','dev_approved','dev_approved_by','qa_test_script_approved','migration_approved_by','qa_test_approved_by']

}
val = bus_rule_dict.get('prod')
print(val)
# # Serializing json 
# json_object = json.dumps(bus_rule_dict, indent = 4)
  
# # Writing to sample.json
# with open("bus_rules.json", "w") as outfile:
#     outfile.write(json_object)
    

['int_test_approved_by', 'dev_approved', 'dev_approved_by', 'qa_test_script_approved', 'migration_approved_by', 'qa_test_approved_by']


In [12]:
df_qa=df.loc[df['ace_migration_status']=='Ready for QA migration']
df_qc=df.loc[df['ace_migration_status']=='Ready for QC migration']
df_ste=df.loc[df['ace_migration_status']=='Ready for STE migration']
df_prod=df.loc[df['ace_migration_status']=='Ready for PROD migration']
#df_qc

# function to create a dataframe on basis of lifecycle
def create_df(org_df, lifecyle):
    for i in lifecyle:
        comp_life = "Ready for " + i.upper() + " migration"
        #print(comp_life)
        mask = org_df['ace_migration_status'] == comp_life
        if any(mask) == True:
            new_df = org_df.loc[mask]
            return new_df
        else:
            print("There is no dataframe for this lifecycle")   
df_qa1 = create_df(df, lifecyle=['qc', 'qa', 'prod', 'ste'])
df_qa1

There is no dataframe for this lifecycle


,key,issue_id,issue_parent_ticket,issue_parent_ticket_summary,issue_rest_url,assignee,reporter,status,issue_type,resolution_description,...,service_name,mig_req_url,dev_approved_by,qa_test_approved_by,int_test_approved_by,dev_approved,new_docker_secrets,qa_test_script_approved,ace_migration_status,migration_approved_by
8,ADAMIGR-1473,136487,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Scott Holze,Scott Holze,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-personbenefitsprofile.yml,https://jira.alight.com/browse/MIG-3152,None,None,Tim Razik,None,No,None,Ready for QA migration,Not Populated
9,ADAMIGR-1467,136462,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Jeff Baisden,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-logonperson\nchannel-usagelog\nchannel...,https://alightdevelopmentandit.atlassian.net/b...,Jeff Baisden,None,Jarett 'J-Man' Shallcross,Yes,Yes,None,Ready for QA migration,Not Populated
11,ADAMIGR-1437,134372,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Jeff Baisden,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-hminquirymobilewidgets,https://alightdevelopmentandit.atlassian.net/b...,Jeff Baisden,None,Jarett 'J-Man' Shallcross,Yes,No,None,Ready for QA migration,Not Populated
12,ADAMIGR-1425,134167,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Jeff Baisden,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-mobiledatacardwidgets,https://alightdevelopmentandit.atlassian.net/b...,Jeff Baisden,None,Jarett 'J-Man' Shallcross,Yes,No,None,Ready for QA migration,Not Populated
13,ADAMIGR-1419,134160,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Jeff Baisden,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-device,https://alightdevelopmentandit.atlassian.net/b...,Jeff Baisden,None,Jarett 'J-Man' Shallcross,Yes,No,None,Ready for QA migration,Not Populated
14,ADAMIGR-1413,134154,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Jeff Baisden,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-mobiledctoolswidgets,https://alightdevelopmentandit.atlassian.net/b...,Jeff Baisden,None,Jarett 'J-Man' Shallcross,Yes,No,None,Ready for QA migration,Not Populated
15,ADAMIGR-1407,134145,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Jeff Baisden,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-mobilehealthtoolswidgets,https://alightdevelopmentandit.atlassian.net/b...,Jeff Baisden,None,Jarett 'J-Man' Shallcross,Yes,No,None,Ready for QA migration,Not Populated
16,ADAMIGR-1401,134139,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Jeff Baisden,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-dbinquirymobilewidgets,https://alightdevelopmentandit.atlassian.net/b...,Jeff Baisden,None,Jarett 'J-Man' Shallcross,Yes,No,None,Ready for QA migration,Not Populated
17,ADAMIGR-1395,134133,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Jeff Baisden,In Progress,Migration Request,This issue is being actively worked on at the ...,...,channel-mobileclientconfiguration,https://alightdevelopmentandit.atlassian.net/b...,Jeff Baisden,None,Jarett 'J-Man' Shallcross,Yes,No,None,Ready for QA migration,Not Populated
18,ADAMIGR-1389,134127,ADAMIGR-33,ACE Release 2.4.1,https://alightdevelopmentandit.atlassian.net/r...,Automation for Jira,Jeff Baisden,In Progress,Migration Request,This issue i

In [13]:
def create_df_with_busrule(org_df, bus_rules, lifecycle=""):
    with open(bus_rules, "r") as json_file:
        bus_rule_dict =json.load(json_file)
    
    if lifecycle in bus_rule_dict.keys():
        
        comp_life = "Ready for " + lifecycle.upper() + " migration"
        #print(comp_life)

        mask = org_df['ace_migration_status'] == comp_life
        if any(mask) == True:
            new_df = org_df.loc[mask]
            
            def validate_row_value(df_row):
                val_list = bus_rule_dict.get(lifecycle)
                if val_list is not None:
                    # print(val_list)
                    for row_name in val_list:
                        if 'None' in df_row[row_name]:
                            return False          
                    return True
                else:
                    print('No business rules defined for this lifecycle')
            new_df['validate_ticket'] = new_df.apply(lambda row: validate_row_value(row), axis=1)
            return new_df
            
        else:
            print("This lifecyle does'nt match with any row of dataframe")
    else:
        print("This business rule does not match with any lifecycle")

prod_df = create_df_with_busrule(df, bus_rules='bus_rules.json', lifecycle='qc')
prod_df

This lifecyle does'nt match with any row of dataframe


In [14]:
# function to read externalize json business rules file
def valid(bus_rul):
    with open(bus_rul, "r") as json_file:
        bus_rul_dict =json.load(json_file)
    return bus_rule_dict
rul = valid(bus_rul='bus_rules.json')
#print(rul)

In [15]:
with open("bus_rules.json", "r") as json_file:
    bus_rule_dict = json.load(json_file)


def validate_row_value(df_row, bus_rule_dict, lifecycle='qc'):
    val_list = bus_rule_dict.get(lifecycle)
    if val_list is not None:
        # print(val_list)
        for row_name in val_list:
            if 'None' in df_row[row_name]:
                return False          
        return True
    else:
        print('No business rules defined for this lifecycle')

df_qc['validate_ticket'] = df_qc.apply(lambda row: validate_row_value(row, bus_rule_dict, lifecycle='qc'), axis=1)
df_qc

ValueError: Expected a 1D array, got an array with shape (0, 24)

In [ ]:
from jira_ticket_module import jira_df
demo_df = jira_df()
demo_df


ModuleNotFoundError: No module named 'jira_ticket_module'